<font size="5">Ingest Time Series - GHG total without LULUCF, in kt CO₂ equivalent into Trino pipeline</font>


In [1]:
# 'capture' magic prevents long outputs from spamming your notebook
#%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib

Load Environment Variables

In [3]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [4]:
# use a catalog that is configured for iceberg
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sandbox'
ingest_table = 'pcaf_unfcc_without_lulucf'

In [5]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

trino_bucket = osc.attach_s3_bucket("S3_DEV")

/opt/app-root/lib64/python3.8/site-packages/sqlalchemy_trino/dialect.py:259: SAWarning: Dialect trino:rest will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  res = connection.execute(sql.text(query)).scalar()


In [6]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

Open a Trino connection using JWT for authentication

In [7]:
# make sure schema exists, or table creation below will fail in weird ways
sql = f"""
create schema if not exists {ingest_catalog}.{ingest_schema}
"""
qres = engine.execute(sql)
print(qres.fetchall())

[(True,)]


In [8]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('default',)
('demo_dv',)
('iceberg_demo',)
('information_schema',)
('pcaf_sovereign_footprint',)
('sandbox',)


In [9]:
pip install country_converter --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 KB 144.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for country_converter: filename=country_converter-0.7.4-py3-none-any.whl size=53542 sha256=94cd40211c34a2d758dd683cb05b729f7f0d3e9b63e7c9a6667c835138944f4a
  Stored in directory: /tmp/pip-ephem-wheel-cache-5vo3zno8/wheels/fa/46/93/aa5e6b53178cc2356f30207643e869b03b7bc83e78a1b589bf
Successfully built country_converter
Note: you may need to restart the kernel to use updated packages.


Load PCAF_UNFCC_WITHOUT_LULUCF

In [10]:
import pandas as pd

ticker_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'PCAF-sovereign-footprint/UNFCC/Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx')
ticker_file.download_file(f'/tmp/Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx')
%run TransposeXLS.py --config UNFCC_without_LULUCF.ini --output=UNFCC_without_LULUCF.csv 
df = pd.read_csv('UNFCC_without_LULUCF.csv')
df= df[['rec_source','data_provider','country_iso_code','country_name','validity_date','attribute','value','unit']].dropna(subset=['value'])
#df["country_iso_code"]='###'
df = df.convert_dtypes()


European Union (Convention) not found in regex
European Union (KP) not found in regex
nan not found in ISO3
Source: UNFCCC GHG Data Interface not found in regex
Note 1: The reporting and review requirements for GHG inventories are different for Annex I and non-Annex I Parties. The definition format of data for emissions/removals from the forestry sector is different for Annex I and non-Annex I Parties. not found in regex
Note 2: Base year data in the data interface relate to the base year under the Climate Change Convention (UNFCCC).  The base year under the Convention is defined slightly different than the base year under the Kyoto Protocol.  An exception is made for European Union (KP) whereby the base year under the Kyoto Protocol is displayed. not found in regex
Note 3: – means "No data available" not found in regex
Note 4: Data displayed on the data interface are "as received" from Parties. The publication of Party submissions on this website does not imply the expression of any o

UNFCC_without_LULUCF.ini
file_list:
['/tmp/Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx']
/tmp/Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx
2
xls
/tmp/Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalen
['0', '2']
                                                Party     Base year  \
0                                           Australia  4.236722e+05   
1                                             Austria  7.842049e+04   
2                                             Belarus  1.391520e+05   
3                                             Belgium  1.457195e+05   
4                                            Bulgaria  1.148010e+05   
5                                              Canada  6.015237e+05   
6                                             Croatia  3.138729e+04   
7                                              Cyprus  5.571026e+03   
8                                             Czechia  1.970718e+05   
9                                  

In [11]:
import osc_ingest_trino as osc
columnschema = osc.create_table_schema_pairs(df) 

sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())



drop table if exists osc_datacommons_dev.sandbox.pcaf_unfcc_without_lulucf

[(True,)]


In [12]:
tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['country_iso_code']
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())


create table if not exists osc_datacommons_dev.sandbox.pcaf_unfcc_without_lulucf(
    rec_source varchar,
    data_provider varchar,
    country_iso_code varchar,
    country_name varchar,
    validity_date bigint,
    attribute varchar,
    value double,
    unit varchar
) with (
    format = 'ORC',
    partitioning = array['country_iso_code']
)

[(True,)]


In [13]:
# Delete all data from our db, so we start with empty table
sql=f"""
delete from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
qres = engine.execute(sql)
print(qres.fetchall())

[(None,)]


In [14]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
pd.read_sql(sql, engine)


,rec_source,data_provider,country_iso_code,country_name,validity_date,attribute,value,unit


In [15]:
print(ingest_catalog)
#df=df.drop(df[df.country_name=="cote d'ivoire"].index)
df.to_sql(ingest_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 1000, verbose = True))

osc_datacommons_dev
constructed fully qualified table name as: "sandbox.pcaf_unfcc_without_lulucf"
inserting 1000 records
  ('Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx', 'UNFCC', 'AUS', 'Australia', 1990, 'Time Series - GHG total without LULUCF, in kt CO₂ equivalent', 423672.2245458623, 'kt CO₂ e')
  ('Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx', 'UNFCC', 'AUT', 'Austria', 1990, 'Time Series - GHG total without LULUCF, in kt CO₂ equivalent', 78420.48531347305, 'kt CO₂ e')
  ('Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx', 'UNFCC', 'BLR', 'Belarus', 1990, 'Time Series - GHG total without LULUCF, in kt CO₂ equivalent', 139151.98563558093, 'kt CO₂ e')
  ...
  ('Time_Series_GHG_total_without_LULUCF_in_kt_CO₂_equivalent.xlsx', 'UNFCC', 'DNK', 'Denmark', 2012, 'Time Series - GHG total without LULUCF, in kt CO₂ equivalent', 54837.68373855551, 'kt CO₂ e')
batch insert result: [(1000,)]
inserting 305 records
  ('Time_Series_GHG_total_wi

In [16]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}"""
pd.read_sql(sql, engine)


,rec_source,data_provider,country_iso_code,country_name,validity_date,attribute,value,unit
0,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCC,NOR,Norway,2012,"Time Series - GHG total without LULUCF, in kt ...",53998.113438,kt CO₂ e
1,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCC,NOR,Norway,2013,"Time Series - GHG total without LULUCF, in kt ...",53989.102361,kt CO₂ e
2,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCC,NOR,Norway,2014,"Time Series - GHG total without LULUCF, in kt ...",53863.144040,kt CO₂ e
3,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCC,NOR,Norway,2015,"Time Series - GHG total without LULUCF, in kt ...",54125.114947,kt CO₂ e
4,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCC,NOR,Norway,2016,"Time Series - GHG total without LULUCF, in kt ...",53095.067314,kt CO₂ e
...,...,...,...,...,...,...,...,...
1300,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCC,KAZ,Kazakhstan,2014,"Time Series - GHG total without LULUCF, in kt ...",331205.223792,kt CO₂ e
1301,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCC,KAZ,Kazakhstan,2015,"Time Series - GHG total without LULUCF, in kt ...",341342.705595,kt CO₂ e
1302,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCC,KAZ,Kazakhstan,2016,"Time Series - GHG total without LULUCF, in kt ...",342616.691388,kt CO₂ e
1303,Time_Series_GHG_total_without_LULUCF_in_kt_CO₂...,UNFCC,KAZ,Kazakhstan,2017,"Time Series - GHG total without LULUCF, in kt ...",364180.033760,kt CO₂ e
